In [ ]:
 # 1 GPU on this machine
%env SM_NUM_GPUS=1
# Where to save the model
%env SM_MODEL_DIR=/tmp/model
# Where the training data is
%env SM_CHANNEL_TRAINING=data
# Where the validation data is
%env SM_CHANNEL_VALIDATION=data

!python train_sagemaker.py --epochs 10

In [ ]:
tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.8xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True
                         )

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(10, 100),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    #'batch-size':    IntegerParameter(256, 1024),
    'cluster_radius': ContinuousParameter(0.001, 0.8),
    'cluster_meanActive':IntegerParameter(2, 100),
    'cluster_shift_meanActive':IntegerParameter(50, 500),
    'cluster_npoint_thre':IntegerParameter(2, 100)
}


objective_metric_name = 'AP_Plant'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'AP_Plant', 'Regex': 'AP_Plant: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=20,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [ ]:
 tuner.fit({'training': training_input_path, 'validation': validation_input_path})